In [1]:
from eruption_forecast.utils import compute_seed_eruption_probability, label_id_to_datetime
import numpy as np
import pandas as pd

In [2]:
label_csv = r"D:\Projects\eruption-forecast\output\VG.OJN.00.EHZ\predictions\features\future_labels_2025-07-28_2025-08-04_ws-10-minutes.csv"

In [3]:
trained_models = r"D:\Projects\eruption-forecast\output\VG.OJN.00.EHZ\trainings\model-only\random-forest-classifier\stratified-k-fold\trained_model_RandomForestClassifier-StratifiedKFold_rs-0_ts-500_top-20.csv"

In [4]:
features_csv = r"D:\Projects\eruption-forecast\output\VG.OJN.00.EHZ\predictions\features\prediction_all_features_2025-07-26-2025-08-04.csv"

In [5]:
label_df = pd.read_csv(label_csv, index_col="id")

In [6]:
features_df = pd.read_csv(features_csv, index_col=0)

In [7]:
features_df.shape

(1152, 3132)

In [55]:
df_models = pd.read_csv(trained_models, index_col=0)
p_eruptions = None

In [56]:
seed_eruption_probabilities: list[np.ndarray] = []
for random_state, row in df_models.iterrows():
    p_eruptions, _ = compute_seed_eruption_probability(
        random_state=random_state,
        features_df=features_df,
        significant_features_csv=row["significant_features_csv"],
        model_filepath=row["trained_model_filepath"],
        debug=True
    )

    seed_eruption_probabilities.append(p_eruptions)

    if random_state == 1:
        break

In [57]:
p_matrix = np.stack(seed_eruption_probabilities, axis=1)

In [58]:
p_mean = p_matrix.mean(axis=1)
p_std = p_matrix.std(axis=1)

In [65]:
pd.DataFrame(confidence)

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1147,1.0
1148,1.0
1149,1.0
1150,1.0


In [59]:
prediction: np.ndarray = (p_mean >= 0.7).astype(int)
prediction

array([0, 0, 0, ..., 0, 0, 0], shape=(1152,))

In [60]:
votes_for_eruption: np.ndarray = (p_matrix >= 0.7).sum(axis=1)

In [61]:
majority_votes = np.where(
    prediction == 1, votes_for_eruption, p_matrix.shape[0] - votes_for_eruption
)

In [64]:
confidence = majority_votes/ p_matrix.shape[0]
confidence

array([1., 1., 1., ..., 1., 1., 1.], shape=(1152,))